Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [26]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import functools
from heapq import heappop, heappush
from collections import deque

In [27]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [28]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def counter(fn):
    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper

@counter
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

## Utility Function

In [29]:
# Computing the manhattan distance, that is the distance between the state and the goal
def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int:
    distance = 0
    for num in range(1, PUZZLE_DIM**2): 
        x1, y1 = np.where(state == num)
        x2, y2 = np.where(goal == num)
        distance += abs(x1 - x2) + abs(y1 - y2)
    return distance

## A-star (A*) Algorithm

In [30]:
def a_star(initial_state: np.ndarray, goal_state: np.ndarray) -> list['Action']:
    # initializing the frontier as (heuristic, cost, state, path)
    frontier = [(manhattan_distance(initial_state, goal_state), 0, tuple(initial_state.flatten()), [])]
    visited = set()

    while frontier:
        # get the state with the lowest heuristic from the frontier
        manh, cost, state_tuple, path = heappop(frontier)
        state = np.array(state_tuple).reshape(PUZZLE_DIM, PUZZLE_DIM)
        
        # return the path if the goal is reached
        if np.array_equal(state, goal_state):
            return path
        
        # if the state_tuple is already visited, skip it
        if state_tuple in visited:
            continue

        # add the current state_tuple in the visited set
        visited.add(state_tuple)
        
        # for each action available from the current state
        for action in available_actions(state):
            next_state = do_action(state, action)
            new_path = path + [action]
            new_cost = cost + 1
            # the new heuristic is the manhattan distance to goal + the new cost
            new_manh = manhattan_distance(next_state, goal_state) + new_cost
            # add the new state to the frontier
            heappush(frontier, (new_manh, new_cost, tuple(next_state.flatten()), new_path))
    
    # return None if the goal is not reachable
    return None

## Breadth-First Search (BFS) Algorithm

In [31]:
def bfs(start, goal) -> list['Action']:
    # initializing the frontier as (state, path)
    frontier = deque([(start, [])])
    visited = set()
    
    while frontier:
        # get the first state from the frontier
        current, path = frontier.popleft()
        
        # return the path if the goal is reached
        if np.array_equal(current, goal):
            return path
        
        # if the state_tuple is already visited, skip it
        if current is visited:
            continue
        
        # add the current state_tuple in the visited set
        visited.add(current.tobytes())
        
        # for each action available from the current state
        for act in available_actions(current):
            next_state = do_action(current, act)
            # add the new state if has not been visited
            if next_state.tobytes() not in visited:
                frontier.append((next_state, path + [act]))

    # return None if the goal is not reachable
    return None


### Generating the goal and randomizing the starting state

In [32]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
goal = state.copy()
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

### Creating a simple state

In [33]:
test_state = goal.copy()
tmp = test_state[2,1]
test_state[2,1] = test_state[2,2]
test_state[2,2] = tmp

### Testing A* with a simple state (only one action is needed)

In [34]:
do_action.calls = 0
path = a_star(test_state, goal)
print(f'Starting state: {test_state}')
print(f'Path: {path}')
print(f'Goal: {goal}')
print(f'Action count: {do_action.calls}')

Starting state: [[1 2 3]
 [4 5 6]
 [7 0 8]]
Path: [Action(pos1=(2, 1), pos2=(2, 2))]
Goal: [[1 2 3]
 [4 5 6]
 [7 8 0]]
Action count: 3


### Testing A* with a real state (more than one action are needed)

In [35]:
do_action.calls = 0
path = a_star(state, goal)
print(f'Starting state: {state}')
print(f'Path: {path}')
print(f'Path length: {len(path)}')
print(f'Goal: {goal}')
print(f'Action count: {do_action.calls}')

Starting state: [[2 5 4]
 [3 8 1]
 [6 7 0]]
Path: [Action(pos1=(2, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(0, 0)), Action(pos1=(0, 0), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(0, 0)), Action(pos1=(0, 0), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 2)

### Testing BFS (Breadth-First Search) with a simple state (only one action is needed)

In [36]:
do_action.calls = 0
path = bfs(test_state, goal)
print(f'Starting state: {test_state}')
print(f'Path: {path}')
print(f'Goal: {goal}')
print(f'Action count: {do_action.calls}')

Starting state: [[1 2 3]
 [4 5 6]
 [7 0 8]]
Path: [Action(pos1=(2, 1), pos2=(2, 2))]
Goal: [[1 2 3]
 [4 5 6]
 [7 8 0]]
Action count: 9


### Testing BFS (Breadth-First Search) with a real state (more than one action are needed)

In [37]:
do_action.calls = 0
path = bfs(state, goal)
print(f'Starting state: {state}')
print(f'Path: {path}')
print(f'Path length: {len(path)}')
print(f'Goal: {goal}')
print(f'Action count: {do_action.calls}')

Starting state: [[2 5 4]
 [3 8 1]
 [6 7 0]]
Path: [Action(pos1=(2, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 0)), Action(pos1=(2, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(2, 1)), Action(pos1=(2, 1), pos2=(2, 2)), Action(pos1=(2, 2), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(0, 2)), Action(pos1=(0, 2), pos2=(0, 1)), Action(pos1=(0, 1), pos2=(0, 0)), Action(pos1=(0, 0), pos2=(1, 0)), Action(pos1=(1, 0), pos2=(1, 1)), Action(pos1=(1, 1), pos2=(1, 2)), Action(pos1=(1, 2), pos2=(2, 2)